In [45]:
import os
from google.oauth2 import service_account
from google.cloud import bigquery

# load service account file
cred_file = os.path.expanduser("~/.creds/dsa-deb-sa.json")      
creds = service_account.Credentials.from_service_account_file(cred_file)

# create a connection
project_id = 'deb-01-372116'
client = bigquery.Client(credentials=creds, project=project_id)

print("Successfully created a BiqQuery client")
print(f"Project: {client.project} \n")

#lists the data sets in project: 'deb-01-372116', 'plants' is infact listed!
print("Listing datasets in deb-01-372116:")
for dataset in client.list_datasets():
    print(f"full_name: `{dataset.full_dataset_id}`")
print(f"\n")

#lists the tables in the plants dataset
dataset_id = 'deb-01-372116.plants'
tables = client.list_tables(dataset_id)
print("Tables contained in '{}':".format(dataset_id))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Successfully created a BiqQuery client
Project: deb-01-372116 

Listing datasets in deb-01-372116:
full_name: `deb-01-372116:airline_test`
full_name: `deb-01-372116:fit_bit`
full_name: `deb-01-372116:mls_salaries`
full_name: `deb-01-372116:my_vinyls`
full_name: `deb-01-372116:plants`
full_name: `deb-01-372116:sf_bikeshare`


Tables contained in 'deb-01-372116.plants':
deb-01-372116.plants.flower_shop


In [46]:
#----------------------fitbit data----------
import pandas as pd
#Read files
hourly_steps_file = "./data/hourlySteps_merged.csv"
hourly_steps = pd.read_csv(hourly_steps_file, header=0)

hourly_calories_file = './data/hourlyCalories_merged.csv'
hourly_calories = pd.read_csv(hourly_calories_file, header=0)

#merge data sets
steps_cal_df = pd.merge(hourly_steps, hourly_calories, on=['ActivityHour', 'Id'], how='left')
steps_cal_df['ActivityHour']=pd.to_datetime(steps_cal_df["ActivityHour"], format="%m/%d/%Y %I:%M:%S %p", utc=True)

#convert acivity hour to readable datetime object
steps_cal_df.drop_duplicates(ignore_index=True)

#write to csv file
steps_cal_df.to_csv("./data/hourly_calories", index=False)
steps_cal_df.head()

# Construct a BigQuery client object to upload csv to google cloud console
client = bigquery.Client()


job_config = bigquery.LoadJobConfig(
    source_format="CSV",
    autodetect=True,
    create_disposition="CREATE_IF_NEEDED",
    write_disposition="WRITE_TRUNCATE",
)

table_id = "deb-01-372116.fit_bit.hourly_calories" #need to create the table first in the console
file_name = "./data/hourly_calories" #where the data is coming from
with open(file_name, "rb") as fp:
    job = client.load_table_from_file(fp, table_id, job_config=job_config)

job.result()

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

,Id,ActivityHour,StepTotal,Calories
0,1503960366,2016-04-12 00:00:00+00:00,373,81
1,1503960366,2016-04-12 01:00:00+00:00,160,61
2,1503960366,2016-04-12 02:00:00+00:00,151,59
3,1503960366,2016-04-12 03:00:00+00:00,0,47
4,1503960366,2016-04-12 04:00:00+00:00,0,48
